### Extract data

In [2]:
import pandas as pd 

df1 = pd.read_csv("order_region_a(in).csv") # location-a
df2 = pd.read_csv("order_region_b(in).csv") # location-b

print(df1.shape, df2.shape)

(44494, 6) (44494, 6)


In [5]:
df1.columns, df2.columns

(Index(['OrderId', 'OrderItemId', 'QuantityOrdered', 'ItemPrice',
        'PromotionDiscount', 'batch_id'],
       dtype='object'),
 Index(['OrderId', 'OrderItemId', 'QuantityOrdered', 'ItemPrice',
        'PromotionDiscount', 'batch_id'],
       dtype='object'))

Schema:

OrderId: Order's id

OrderItemId: Item ids of the purchase orders.

QuantityOrdered: Number of items Ordered for an order.

ItemPrice: Price of each item in INR.

PromotionDiscount: Discount of an Order.

#### Applying business-rules

Business Rules:
1. Combine the data from both regions into a single table.
2. Add a column total_sales which is calculated as QuantityOrdered * ItemPrice.
3. Add a column region to identify the region of the sales record (A or B).
4. Ensure that there are no duplicate entries based on OrderId.
5. Add a new column net_sale, calculated as total_sales - PromotionDiscount.
6. Exclude orders where the total sales amount is negative or zero after applying discounts.
7. Load the transformed data into a the database of your choice.

### Transform Data 

In [16]:
# 3. Add a column region to identify the region of the sales record (A or B).

df1['region'] = 'A' 
df2['region'] = 'B' 

df1.columns, df2.columns

(Index(['OrderId', 'OrderItemId', 'QuantityOrdered', 'ItemPrice',
        'PromotionDiscount', 'batch_id', 'region'],
       dtype='object'),
 Index(['OrderId', 'OrderItemId', 'QuantityOrdered', 'ItemPrice',
        'PromotionDiscount', 'batch_id', 'region'],
       dtype='object'))

In [17]:
# 1. Combine the data from both regions into a single table. 

df = pd.concat([df1, df2], axis=0, ignore_index=True)
df.shape

(88988, 7)

In [18]:
df

,OrderId,OrderItemId,QuantityOrdered,ItemPrice,PromotionDiscount,batch_id,region
0,171-0001135-1657958,1.116890e+13,1,949.0,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10""}",359,A
1,171-0001497-9165123,1.976030e+13,1,699.0,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10.1""}",1135,A
2,171-0002127-1363507,5.949760e+12,1,399.0,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10""}",297,A
3,171-0002370-0601169,5.757190e+13,1,499.0,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10.1""}",114,A
4,171-0004526-2028348,3.385130e+13,1,1699.0,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10""}",764,A
...,...,...,...,...,...,...,...
88983,403-9398294-9729165,2.595270e+13,1,899.0,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10.1""}",729,B
88984,403-9398429-3695549,5.373600e+13,1,499.0,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10""}",151,B
88985,403-9399318-8267565,7.727720e+12,1,499.0,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10.1""}",54,B
88986,403-9400326-2753965,2.029820e+13,1,499.0,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10""}",1401,B


In [90]:
# 2. Add a column total_sales which is calculated as QuantityOrdered * ItemPrice.

df['total_sales'] = df['QuantityOrdered'] * df['ItemPrice']  
df.head()

,OrderId,OrderItemId,QuantityOrdered,ItemPrice,PromotionDiscount,batch_id,region,total_sales
0,171-0001135-1657958,1.116890e+13,1,949.0,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10""}",359,A,949.0
1,171-0001497-9165123,1.976030e+13,1,699.0,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10.1""}",1135,A,699.0
2,171-0002127-1363507,5.949760e+12,1,399.0,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10""}",297,A,399.0
3,171-0002370-0601169,5.757190e+13,1,499.0,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10.1""}",114,A,499.0
4,171-0004526-2028348,3.385130e+13,1,1699.0,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10""}",764,A,1699.0


In [91]:
df[df['QuantityOrdered'] > 1].head()    # to check the above calc

,OrderId,OrderItemId,QuantityOrdered,ItemPrice,PromotionDiscount,batch_id,region,total_sales
53,171-0034857-3794740,3.586060e+13,2,1198.0,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10.1""}",188,A,2396.0
63,171-0039618-4949171,2.855330e+12,2,558.0,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10.1""}",165,A,1116.0
112,171-0075785-1220351,2.640650e+13,2,5998.0,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10""}",116,A,11996.0
179,171-0109070-3901902,3.413900e+13,2,3198.0,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10.1""}",721,A,6396.0
294,171-0180848-2574766,3.179560e+13,3,1347.0,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10""}",1241,A,4041.0


In [99]:
# 4. Ensure that there are no duplicate entries based on OrderId.

cleaned_df = df.drop_duplicates(subset='OrderId', keep='first').reset_index(drop=True)
cleaned_df.shape

(41107, 8)

In [101]:
cleaned_df['region'].unique()  # what ??????, all region-B are gone?????

array(['A'], dtype=object)

In [94]:
cleaned_df.dtypes   # to dtype of PromotionDiscount (json or string)

index                  int64
OrderId               object
OrderItemId          float64
QuantityOrdered        int64
ItemPrice            float64
PromotionDiscount     object
batch_id               int64
region                object
total_sales          float64
dtype: object

In [102]:
# 5. Add a new column net_sale, calculated as total_sales - PromotionDiscount.

def extract_amount(obj):     # to extract amount from the obj
    obj = obj.strip("{} ")
    items = obj.split(',')
    amount = items[1].strip().split(':')
    return float(amount[1].strip('" ')) 

val = extract_amount('{ "CurrencyCode": "INR", "Amount": "10.1"}') 
print(val, type(val))

10.1 <class 'float'>


In [103]:
cleaned_df['PromotionDiscount'].apply(extract_amount)   # working fine

0        10.0
1        10.1
2        10.0
3        10.1
4        10.0
         ... 
41102    10.1
41103    10.0
41104    10.1
41105    10.0
41106    10.1
Name: PromotionDiscount, Length: 41107, dtype: float64

In [104]:
cleaned_df['DiscountAmount'] = cleaned_df['PromotionDiscount'].apply(extract_amount)
cleaned_df.head()

,OrderId,OrderItemId,QuantityOrdered,ItemPrice,PromotionDiscount,batch_id,region,total_sales,DiscountAmount
0,171-0001135-1657958,1.116890e+13,1,949.0,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10""}",359,A,949.0,10.0
1,171-0001497-9165123,1.976030e+13,1,699.0,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10.1""}",1135,A,699.0,10.1
2,171-0002127-1363507,5.949760e+12,1,399.0,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10""}",297,A,399.0,10.0
3,171-0002370-0601169,5.757190e+13,1,499.0,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10.1""}",114,A,499.0,10.1
4,171-0004526-2028348,3.385130e+13,1,1699.0,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10""}",764,A,1699.0,10.0


In [105]:
non_float_rows = cleaned_df[~cleaned_df['total_sales'].apply(lambda x: isinstance(x, float))]   # checking for non-float or non-valid rows
print(non_float_rows)

Empty DataFrame
Columns: [OrderId, OrderItemId, QuantityOrdered, ItemPrice, PromotionDiscount, batch_id, region, total_sales, DiscountAmount]
Index: []


In [106]:
non_float_rows = cleaned_df[~cleaned_df['DiscountAmount'].apply(lambda x: isinstance(x, float))]   # checking for non-float or non-valid rows
print(non_float_rows)

Empty DataFrame
Columns: [OrderId, OrderItemId, QuantityOrdered, ItemPrice, PromotionDiscount, batch_id, region, total_sales, DiscountAmount]
Index: []


In [107]:
cleaned_df['net_sale'] = cleaned_df['total_sales'] - cleaned_df['DiscountAmount'] 
cleaned_df.head()

,OrderId,OrderItemId,QuantityOrdered,ItemPrice,PromotionDiscount,batch_id,region,total_sales,DiscountAmount,net_sale
0,171-0001135-1657958,1.116890e+13,1,949.0,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10""}",359,A,949.0,10.0,939.0
1,171-0001497-9165123,1.976030e+13,1,699.0,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10.1""}",1135,A,699.0,10.1,688.9
2,171-0002127-1363507,5.949760e+12,1,399.0,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10""}",297,A,399.0,10.0,389.0
3,171-0002370-0601169,5.757190e+13,1,499.0,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10.1""}",114,A,499.0,10.1,488.9
4,171-0004526-2028348,3.385130e+13,1,1699.0,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10""}",764,A,1699.0,10.0,1689.0


In [108]:
cleaned_df.shape

(41107, 10)

In [109]:
# 5. Exclude orders where the total sales amount is negative or zero after applying discounts.

invalid_sale_df = cleaned_df[cleaned_df['net_sale'] <= 0]
invalid_sale_df.shape   # total 55 such rows

(55, 10)

In [110]:
final_df = cleaned_df[cleaned_df['net_sale'] > 0] 
final_df.shape

(41052, 10)

In [111]:
41107 - 55    #validated

41052

### Load Data

In [112]:
# 7. Load the transformed data into a the database of your choice. 

import sqlite3

# Connect to (or create) an SQLite database file
conn = sqlite3.connect("orders_database.db")

conn

In [113]:
# Load the DataFrame into a SQL table
final_df.to_sql("orders", conn, if_exists="replace", index=False)  # replace if exists before 

41052

In [114]:
# verify the table exitence 

result_df = pd.read_sql_query("SELECT * FROM orders", conn)

result_df.shape

(41052, 10)

### SQL Queries
a. Count the total number of records.

b. Find the total sales amount by region


c. Find the average sales amount per transactio

.
d. Ensure there are no duplicate OrderId values.

In [115]:
# a. Count the total number of records 

total_orders = pd.read_sql_query("SELECT count(*) as total_final_orders FROM orders", conn) 
total_orders

,total_final_orders
0,41052


In [119]:
# b. Find the total sales amount by region

query = "select region, sum(total_sales) as total_sales from orders group by 1" 
total_sales_per_region = pd.read_sql_query(query, conn) 
total_sales_per_region.head()

,region,total_sales
0,A,34570984.52


In [120]:
# c. Find the average sales amount per transaction 

query = "select OrderId, avg(net_sale) as avg_sales from orders group by 1" 
avg_sales_per_trans = pd.read_sql_query(query, conn) 
avg_sales_per_trans.head()

,OrderId,avg_sales
0,171-0001135-1657958,939.0
1,171-0001497-9165123,688.9
2,171-0002127-1363507,389.0
3,171-0002370-0601169,488.9
4,171-0004526-2028348,1689.0


In [121]:
# . d. Ensure there are no duplicate OrderId values.

query = "select OrderId, count(*) as cnt from orders group by 1 having count(*) > 1" 
duplicates = pd.read_sql_query(query, conn) 
duplicates.head()

,OrderId,cnt


In [78]:
# close the connection 
conn.close() 